In [7]:
import pandas as pd
import numpy as np
import os
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
from utils import train_random_forest, train_stacker,train_xgboost, cap_outliers

In [8]:
prefecture_code = pd.read_csv("japan_housing_data/prefecture_code.csv")
prefecture_code.head()

,Code,JpName,EnName
0,1,北海道,Hokkaido
1,2,青森県,Aomori
2,3,岩手県,Iwate
3,4,宮城県,Miyagi
4,5,秋田県,Akita


In [9]:
df = pd.DataFrame()

data_dir = "japan_housing_data/trade_prices"

for f in os.listdir(data_dir):
    df_prefecture = pd.read_csv(os.path.join(data_dir, f), low_memory=False)
    df_prefecture["Code"] = int(f.split(".")[0])
    df = pd.concat([df, df_prefecture])

df.head()

,No,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,NearestStation,TimeToNearestStation,MinTimeToNearestStation,...,Breadth,CityPlanning,CoverageRatio,FloorAreaRatio,Period,Year,Quarter,Renovation,Remarks,Code
0,1,Residential Land(Land Only),Residential Area,40101,Fukuoka Prefecture,"Moji Ward,Kitakyushu City",Aobadai,Moji,30-60minutes,30.0,...,4.0,Category I Exclusively Low-story Residential Zone,50.0,80.0,4th quarter 2017,2017,4,NaN,NaN,40
1,2,Residential Land(Land Only),Residential Area,40101,Fukuoka Prefecture,"Moji Ward,Kitakyushu City",Oaza Ikawa,Komorie,30-60minutes,30.0,...,4.0,Urbanization Control Area,NaN,NaN,1st quarter 2018,2018,1,NaN,NaN,40
2,3,Residential Land(Land Only),Residential Area,40101,Fukuoka Prefecture,"Moji Ward,Kitakyushu City",Oaza Ikawa,Moji,1H-1H30,60.0,...,4.0,Urbanization Control Area,NaN,NaN,4th quarter 2017,2017,4,NaN,NaN,40
3,4,Residential Land(Land Only),Residential Area,40101,Fukuoka Prefecture,"Moji Ward,Kitakyushu City",Oaza Ikawa,Moji,1H-1H30,60.0,...,35.0,Urbanization Control Area,70.0,200.0,4th quarter 2016,2016,4,NaN,NaN,40
4,5,Residential Land(Land Only),Residential Area,40101,Fukuoka Prefecture,"Moji Ward,Kitakyushu City",Oaza Ikawa,Moji,1H-1H30,60.0,...,NaN,Urbanization Control Area,70.0,200.0,3rd quarter 2016,2016,3,NaN,NaN,40


In [10]:
df = pd.merge(left=df, right=prefecture_code, left_on="Code", right_on="Code", how="left")
df.head()

,No,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,NearestStation,TimeToNearestStation,MinTimeToNearestStation,...,CoverageRatio,FloorAreaRatio,Period,Year,Quarter,Renovation,Remarks,Code,JpName,EnName
0,1,Residential Land(Land Only),Residential Area,40101,Fukuoka Prefecture,"Moji Ward,Kitakyushu City",Aobadai,Moji,30-60minutes,30.0,...,50.0,80.0,4th quarter 2017,2017,4,NaN,NaN,40,福岡県,Fukuoka
1,2,Residential Land(Land Only),Residential Area,40101,Fukuoka Prefecture,"Moji Ward,Kitakyushu City",Oaza Ikawa,Komorie,30-60minutes,30.0,...,NaN,NaN,1st quarter 2018,2018,1,NaN,NaN,40,福岡県,Fukuoka
2,3,Residential Land(Land Only),Residential Area,40101,Fukuoka Prefecture,"Moji Ward,Kitakyushu City",Oaza Ikawa,Moji,1H-1H30,60.0,...,NaN,NaN,4th quarter 2017,2017,4,NaN,NaN,40,福岡県,Fukuoka
3,4,Residential Land(Land Only),Residential Area,40101,Fukuoka Prefecture,"Moji Ward,Kitakyushu City",Oaza Ikawa,Moji,1H-1H30,60.0,...,70.0,200.0,4th quarter 2016,2016,4,NaN,NaN,40,福岡県,Fukuoka
4,5,Residential Land(Land Only),Residential Area,40101,Fukuoka Prefecture,"Moji Ward,Kitakyushu City",Oaza Ikawa,Moji,1H-1H30,60.0,...,70.0,200.0,3rd quarter 2016,2016,3,NaN,NaN,40,福岡県,Fukuoka


In [11]:
df.columns

Index(['No', 'Type', 'Region', 'MunicipalityCode', 'Prefecture',
       'Municipality', 'DistrictName', 'NearestStation',
       'TimeToNearestStation', 'MinTimeToNearestStation',
       'MaxTimeToNearestStation', 'TradePrice', 'FloorPlan', 'Area',
       'AreaIsGreaterFlag', 'UnitPrice', 'PricePerTsubo', 'LandShape',
       'Frontage', 'FrontageIsGreaterFlag', 'TotalFloorArea',
       'TotalFloorAreaIsGreaterFlag', 'BuildingYear', 'PrewarBuilding',
       'Structure', 'Use', 'Purpose', 'Direction', 'Classification', 'Breadth',
       'CityPlanning', 'CoverageRatio', 'FloorAreaRatio', 'Period', 'Year',
       'Quarter', 'Renovation', 'Remarks', 'Code', 'JpName', 'EnName'],
      dtype='object')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3906518 entries, 0 to 3906517
Data columns (total 41 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   No                           int64  
 1   Type                         object 
 2   Region                       object 
 3   MunicipalityCode             int64  
 4   Prefecture                   object 
 5   Municipality                 object 
 6   DistrictName                 object 
 7   NearestStation               object 
 8   TimeToNearestStation         object 
 9   MinTimeToNearestStation      float64
 10  MaxTimeToNearestStation      float64
 11  TradePrice                   int64  
 12  FloorPlan                    object 
 13  Area                         int64  
 14  AreaIsGreaterFlag            int64  
 15  UnitPrice                    float64
 16  PricePerTsubo                float64
 17  LandShape                    object 
 18  Frontage                     float64
 19  

In [13]:
df.describe().apply(lambda s: s.apply('{0:.1f}'.format))  

,No,MunicipalityCode,MinTimeToNearestStation,MaxTimeToNearestStation,TradePrice,Area,AreaIsGreaterFlag,UnitPrice,PricePerTsubo,Frontage,TotalFloorArea,TotalFloorAreaIsGreaterFlag,BuildingYear,PrewarBuilding,Breadth,CoverageRatio,FloorAreaRatio,Year,Quarter,Code
count,3906518.0,3906518.0,3315977.0,3228071.0,3906518.0,3906518.0,3906518.0,1384415.0,1384415.0,2537063.0,1317660.0,3906518.0,1869804.0,3906518.0,2685837.0,3191877.0,3191877.0,3906518.0,3906518.0,3906518.0
mean,78354.1,20467.1,22.6,27.4,27605894.6,467.4,0.0,85148.9,281406.5,14.6,175.2,0.0,1995.5,0.0,6.8,61.0,209.0,2012.9,2.5,20.3
std,80173.6,12175.5,23.1,27.7,141745961.4,847.7,0.2,168683.2,557031.6,9.8,256.4,0.1,15.2,0.0,4.6,10.0,107.5,3.7,1.1,12.2
min,1.0,1101.0,0.0,0.0,100.0,10.0,0.0,1.0,2.0,0.1,10.0,0.0,1945.0,0.0,1.0,30.0,50.0,2005.0,1.0,1.0
25%,20780.0,12204.0,9.0,8.0,5000000.0,100.0,0.0,16000.0,55000.0,8.5,95.0,0.0,1984.0,0.0,4.0,60.0,200.0,2010.0,2.0,12.0
50%,48159.0,17210.0,16.0,15.0,14000000.0,185.0,0.0,40000.0,130000.0,12.0,105.0,0.0,1996.0,0.0,6.0,60.0,200.0,2013.0,3.0,17.0
75%,114733.0,28110.0,30.0,29.0,29000000.0,370.0,0.0,91000.0,300000.0,17.0,140.0,0.0,2009.0,0.0,7.0,60.0,200.0,2016.0,4.0,28.0
max,406575.0,47382.0,120.0,120.0,61000000000.0,5000.0,1.0,20000000.0,66000000.0,50.0,2000.0,1.0,2020.0,1.0,99.9,80.0,1300.0,2019.0,4.0,47.0


In [14]:
df['EnName'] = df['EnName'].apply(lambda x: x.rstrip())

In [15]:
missing_percentage = (df.isnull().mean() * 100).sort_values(ascending=False)
print("Missing percentage\n")
print(missing_percentage)

Missing percentage

Remarks                        94.064791
Renovation                     85.578052
FloorPlan                      84.713497
Purpose                        69.722960
TotalFloorArea                 66.270218
PricePerTsubo                  64.561407
UnitPrice                      64.561407
BuildingYear                   52.136301
Use                            51.709579
Structure                      51.041490
Frontage                       35.055643
Breadth                        31.247290
Classification                 30.514719
Direction                      28.901646
LandShape                      28.862788
Region                         28.595209
FloorAreaRatio                 18.293555
CoverageRatio                  18.293555
MaxTimeToNearestStation        17.367052
MinTimeToNearestStation        15.116812
TimeToNearestStation           15.116812
NearestStation                 14.483768
CityPlanning                   13.584118
DistrictName                    0.211

In [16]:
df['MunicipalityCode'] = df['MunicipalityCode'].astype(str)

In [17]:
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col].fillna('unkown', inplace=True)

In [18]:
df_copy = df.copy()

In [ ]:
kyushu_prefectures = ['Fukuoka', 'Saga', 'Nagasaki', 'Kumamoto', 'Oita', 'Miyazaki', 'Kagoshima']
kyushu_df = df_copy[df_copy['EnName'].isin(kyushu_prefectures)]

In [14]:
split_index = int(len(kyushu_df) * 0.85)

kyushu_df = kyushu_df.sort_values(by=['Year', 'Quarter'])

In [15]:
kyushu_df[["Year", "Quarter"]].head(100050)

,Year,Quarter
125306,2005,3
125307,2005,3
125761,2005,3
130110,2005,3
131394,2005,3
...,...,...
87770,2010,3
87771,2010,3
87811,2010,3
87813,2010,3


In [16]:
train_df = kyushu_df.iloc[:split_index]
test_df = kyushu_df.iloc[split_index:]

In [17]:
print(test_df[['Year', 'Quarter']].head(100000))

        Year  Quarter
174524  2017        4
174525  2017        4
174529  2017        4
174558  2017        4
174559  2017        4
...      ...      ...
416754  2019        3
416755  2019        3
416800  2019        3
416801  2019        3
416973  2019        3

[60678 rows x 2 columns]


In [18]:
def cap_outliers(series, lower_percentile=0.01, upper_percentile=0.99):
    lower = series.quantile(lower_percentile)
    upper = series.quantile(upper_percentile)
    return np.clip(series, lower, upper)

In [19]:
kyushu_df.head()

,No,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,NearestStation,TimeToNearestStation,MinTimeToNearestStation,...,CoverageRatio,FloorAreaRatio,Period,Year,Quarter,Renovation,Remarks,Code,JpName,EnName
125306,125307,Residential Land(Land Only),Residential Area,40202,Fukuoka Prefecture,Omuta City,Oaza Iwamoto,Yoshino (Fukuoka),30-60minutes,30.0,...,40.0,60.0,3rd quarter 2005,2005,3,unkown,unkown,40,福岡県,Fukuoka
125307,125308,Residential Land(Land Only),Potential Residential Area,40202,Fukuoka Prefecture,Omuta City,Oaza Iwamoto,Yoshino (Fukuoka),30-60minutes,30.0,...,60.0,200.0,3rd quarter 2005,2005,3,unkown,unkown,40,福岡県,Fukuoka
125761,125762,Residential Land(Land and Building),Residential Area,40203,Fukuoka Prefecture,Kurume City,Aikawamachi,Kurumedaigakumae,8,8.0,...,60.0,200.0,3rd quarter 2005,2005,3,unkown,unkown,40,福岡県,Fukuoka
130110,130111,Residential Land(Land and Building),Residential Area,40217,Fukuoka Prefecture,Chikushino City,Okada,Chikushi,16,16.0,...,40.0,60.0,3rd quarter 2005,2005,3,unkown,unkown,40,福岡県,Fukuoka
131394,131395,Residential Land(Land Only),Residential Area,40219,Fukuoka Prefecture,Onojo City,unkown,Onojo,23,23.0,...,50.0,80.0,3rd quarter 2005,2005,3,unkown,unkown,40,福岡県,Fukuoka


In [20]:
columns_to_cap = ["Area", "TotalFloorArea","FloorAreaRatio"]
for col in columns_to_cap:    
    train_df[col] = cap_outliers(train_df[col])

In [21]:
missing_numerical = train_df.select_dtypes(include=['float64', 'int64']).isna().sum().sort_values(ascending=False)
missing_percentage = (train_df.select_dtypes(include=['float64', 'int64']).isnull().mean() * 100).sort_values(ascending=False)
print("Missing values in numerical variables:")
print(missing_numerical[missing_numerical > 0])
print("\n")
print(missing_percentage[missing_numerical > 0])

Missing values in numerical variables:
TotalFloorArea             251472
UnitPrice                  227993
PricePerTsubo              227993
BuildingYear               225538
Frontage                   157202
Breadth                    141998
FloorAreaRatio             126789
CoverageRatio              126789
MaxTimeToNearestStation    119041
MinTimeToNearestStation    110086
dtype: int64


TotalFloorArea             73.136555
UnitPrice                  66.308069
PricePerTsubo              66.308069
BuildingYear               65.594072
Frontage                   45.719654
Breadth                    41.297817
FloorAreaRatio             36.874526
CoverageRatio              36.874526
MaxTimeToNearestStation    34.621145
MinTimeToNearestStation    32.016729
dtype: float64


In [22]:
columns_to_drop = ['JpName', 'DistrictName', 'UnitPrice','PricePerTsubo', 'No', 
                   'TimeToNearestStation', 'TotalFloorAreaIsGreaterFlag', 'NearestStation', 
                   'Period', 'Year', 'Municipality', 'EnName', 'Prefecture', 
                   'AreaIsGreaterFlag', 'Quarter', 'FrontageIsGreaterFlag'
                   ]
train_df = train_df.drop(columns= columns_to_drop)
test_df = test_df.drop(columns= columns_to_drop)

In [23]:
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer(n_neighbors=5)

numerical_columns = train_df.select_dtypes(include=['number']).columns
train_df_knn = train_df.copy()
test_df_knn = test_df.copy()

train_df_knn[numerical_columns] = knn_imputer.fit_transform(train_df_knn[numerical_columns])
test_df_knn[numerical_columns] = knn_imputer.transform(test_df_knn[numerical_columns])
print(test_df_knn[numerical_columns])


        MinTimeToNearestStation  MaxTimeToNearestStation  TradePrice    Area  \
174524                     36.2                     54.2    730000.0  1500.0   
174525                     65.2                     51.0    190000.0   125.0   
174529                     28.0                     28.0  11000000.0   810.0   
174558                     46.2                     52.2   1100000.0  2200.0   
174559                     72.0                     56.8    490000.0   990.0   
...                         ...                      ...         ...     ...   
416754                     72.0                     78.0   2500000.0  5000.0   
416755                     72.0                     50.8    200000.0  1900.0   
416800                     44.8                     44.8     50000.0  1900.0   
416801                     72.0                     84.0    300000.0  1300.0   
416973                     30.0                     60.0   1600000.0   260.0   

        Frontage  TotalFloorArea  Build

In [24]:
X_train = train_df_knn.drop(columns=["TradePrice"])
y_train = train_df_knn["TradePrice"]
X_test = test_df_knn.drop(columns=["TradePrice"])
y_test = test_df_knn["TradePrice"]

In [25]:
print("Training data shape: ", X_train.shape, y_train.shape)
print("Test data shape: ", X_test.shape, y_test.shape)

Training data shape:  (343839, 24) (343839,)
Test data shape:  (60678, 24) (60678,)


In [26]:
missing_numerical = X_train.select_dtypes(include=['float64', 'int64']).isna().sum().sort_values(ascending=False)
missing_percentage = (X_train.select_dtypes(include=['float64', 'int64']).isnull().mean() * 100).sort_values(ascending=False)
print("Missing values in numerical variables:")
print(missing_numerical[missing_numerical > 0])
print("\n")
print(missing_percentage[missing_numerical > 0])

Missing values in numerical variables:
Series([], dtype: int64)


Series([], dtype: float64)


In [ ]:
rf_result = train_random_forest(X_train.copy(), y_train, X_test.copy(), y_test)
xgb_result = train_xgboost(X_train.copy(), y_train, X_test.copy(), y_test)
stacker_result = train_stacker(X_train.copy(), y_train, X_test.copy(), y_test, rf_result['top_12_features'])


Random Forest R-squared: 0.4491
R2 score Random Forest 0.4491440461805386
XGB Regressor R-squared: 0.4384
R2 score XGBRegressor 0.438353552603867
Stacking Regressor R-squared: 0.5061
R2 score Stacking Regressor 0.5061257198124639


In [ ]:
# Saga
import joblib
preferture = 'Saga'

joblib.dump(rf_result['model'], f"modelsKNN/random_forest_{preferture}.pkl")
joblib.dump(xgb_result['model'], f"modelsKNN/xgboost_{preferture}.pkl")
joblib.dump(stacker_result['model'], f"modelsKNN/stacking_{preferture}.pkl")

joblib.dump(stacker_result['model'], f"best_models/stacking_{preferture}.pkl")


['best_models/stacking_Saga.pkl']